<a href="https://colab.research.google.com/github/adigenova/uohpmd/blob/main/code/OMP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenMP II

## Ejemplos intermedios

In [1]:
!apt install libomp-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libomp-14-dev libomp5-14
Suggested packages:
  libomp-14-doc
The following NEW packages will be installed:
  libomp-14-dev libomp-dev libomp5-14
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 738 kB of archives.
After this operation, 8,991 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp5-14 amd64 1:14.0.0-1ubuntu1.1 [389 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp-14-dev amd64 1:14.0.0-1ubuntu1.1 [347 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libomp-dev amd64 1:14.0-55~exp2 [3,074 B]
Fetched 738 kB in 1s (872 kB/s)
Selecting previously unselected package libomp5-14:amd64.
(Reading database ... 120895 files and directories currently installed.)
Preparing to unpack .../libomp5-14_1%3a14

## Ejemplo 1
cómo calcular la suma de elementos en un arreglo utilizando múltiples hilos:

In [2]:
%%writefile ejem1.c

#include <stdio.h>
#include <omp.h>

int main() {
    int arr[] = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};
    int n = 10;
    int sum = 0;
    //
    #pragma omp parallel for reduction(+:sum)
    for (int i = 0; i < n; i++) {
        sum += arr[i];
        printf("Thread %d: arr[%d] = %d\n", omp_get_thread_num(), i, arr[i]);
    }

    printf("Sum = %d\n", sum);
    return 0;
}


Writing ejem1.c


In [ ]:
!gcc -o ejem1 -fopenmp ejem1.c
%env OMP_NUM_THREADS=8
!./ejem1

## Ejemplo2



In [9]:
%%writefile ejem2.c

#include <stdio.h>
#include <omp.h>

#define N 10000
double data[N];

// Función para calcular el promedio de un subconjunto del arreglo
double calcularPromedio(int start, int end) {
    double sum = 0.0;
    for (int i = start; i < end; i++) {
        sum += data[i];
    }
    return sum / (end - start);
}

int main() {
    // Inicializar el arreglo de datos
    for (int i = 0; i < N; i++) {
        data[i] = i;
    }

    int num_threads = 4; // Número de hilos a utilizar
    double total_average = 0.0;

    #pragma omp parallel num_threads(num_threads)
    {
        int thread_id = omp_get_thread_num();
        int chunk_size = N / num_threads;
        int start = thread_id * chunk_size;
        int end = (thread_id == num_threads - 1) ? N : start + chunk_size;

        double local_average = calcularPromedio(start, end);

        #pragma omp critical
        total_average += local_average;

        printf("Thread %d: Local Average = %f\n", thread_id, local_average);
    }

    total_average /= num_threads;
    printf("Total Average = %f\n", total_average);

    return 0;
}


Overwriting ejem2.c


In [ ]:
!gcc -o ejem2 -fopenmp ejem2.c
%env OMP_NUM_THREADS=8
!./ejem2

## Ejemplo 3
Contando caracteres en paralelo

In [25]:
%%writefile ejem3.c

#include <stdio.h>
#include <string.h>
#include <omp.h>

#define MAX_TEXT_SIZE 10000

// Función para contar la frecuencia de caracteres en un texto
void countCharacterFrequency(const char* text, int* charCount) {
    int chartmp[256] = {0};
    for (int i = 0; text[i] != '\0'; i++) {
        char c = text[i];
        chartmp[c]++;
    }
    //sumamos el resultado en charCount
    #pragma omp critical
    for(int j = 0; j<256; j++){
        charCount[j]+=chartmp[j];
    }

}

int main() {
    char text[MAX_TEXT_SIZE];
    int charCount[256] = {0}; // 256  caracteres asccii
    // Texto
    const char* inputText = "Los recursos en línea ofrecen una variedad de textos en español para mejorar la comprensión y el aprendizaje del idioma. Estos textos pueden ser útiles para estudiantes de diferentes niveles de habilidad.";

    // Divide the text into two segments
    char* segment1 = strncpy(text, inputText, strlen(inputText) / 2);
    char* segment2 = strncpy(text + (strlen(inputText) / 2), inputText + ( strlen(inputText) / 2),  strlen(inputText) / 2);

    #pragma omp parallel
    {
        #pragma omp sections
        {
            #pragma omp section
            {
                countCharacterFrequency(segment1, charCount);
            }
            #pragma omp section
            {
                countCharacterFrequency(segment2, charCount);
            }
        }
    }

    printf("Frecuencia de caracteres:\n");
    for (int i = 0; i < 256; i++) {
        if (charCount[i] > 0) {
            printf("Caracter '%c' aparece %d veces\n", (char)i, charCount[i]);
        }
    }

    return 0;
}


Overwriting ejem3.c


In [ ]:
!gcc -o ejem3 -fopenmp ejem3.c
%env OMP_NUM_THREADS=4
!./ejem3

## Ejemplos Avanzados



Ordenar en paralelo los numeros de un arreglo

In [61]:
%%writefile ejem4.c

#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define ARRAY_SIZE 2000000

void quickSort(int arr[], int left, int right) {
    if (left < right) {
        int pivot = arr[right];
        int i = left - 1;

        for (int j = left; j < right; j++) {
            if (arr[j] < pivot) {
                i++;
                int temp = arr[i];
                arr[i] = arr[j];
                arr[j] = temp;
            }
        }

        int temp = arr[i + 1];
        arr[i + 1] = arr[right];
        arr[right] = temp;

        int partition = i + 1;

        #pragma omp parallel sections
        {
            #pragma omp section
            quickSort(arr, left, partition - 1);
            #pragma omp section
            quickSort(arr, partition + 1, right);
        }
    }
}

int main() {
    int arr[ARRAY_SIZE];

    // Inicializar el arreglo con números aleatorios
    for (int i = 0; i < ARRAY_SIZE; i++) {
        arr[i] = rand() % 10000;
    }

    printf("Arreglo no ordenado:\n");
    for (int i = 0; i < 100; i++) {
        printf("%d ", arr[i]);
    }
    printf("\n");

    // Ordenar el arreglo usando Quick Sort paralelizado
    #pragma omp parallel
    {
        #pragma omp single
        quickSort(arr, 0, ARRAY_SIZE - 1);
    }

    printf("Arreglo ordenado:\n");
    for (int i = 0; i < 100; i++) {
        printf("%d ", arr[i]);
    }
    printf("\n");

    return 0;
}


Overwriting ejem4.c


In [ ]:
!gcc -o ejem4 -fopenmp ejem4.c
%env OMP_NUM_THREADS=4
!time ./ejem4

In [93]:
%%writefile ejem5.c

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <omp.h>
#include <time.h>

#define NUM_TURTLES 5
#define RACE_DISTANCE 40

int race_track[RACE_DISTANCE];

void initializeRaceTrack() {
    for (int i = 0; i < RACE_DISTANCE; i++) {
        race_track[i] = -1; // Inicializar la pista con -1 (sin tortugas)
    }
}

void printRaceTrack() {
    printf("Pista: ");
    for (int i = 0; i < RACE_DISTANCE; i++) {
        if (race_track[i] == -1) {
            printf(".");
        } else {
            printf("%d", race_track[i]);
        }
    }
    printf("\n");
}

int main() {
    //srand(time(NULL));
    int winner = -1;

    initializeRaceTrack();

    #pragma omp parallel
    {
        int turtle_id = omp_get_thread_num();
        int position = -1;

        for (int i = 0; i < RACE_DISTANCE; i++) {
            int move = rand() % 4;
            sleep(move/2);
            #pragma omp critical
            {
                race_track[position] = -1;
                position += move;
                if (position >= RACE_DISTANCE) {
                    position = RACE_DISTANCE - 1;
                    if (winner == -1) {
                        winner = turtle_id;
                    }
                }
                race_track[position] = turtle_id;
                printRaceTrack();
            }
        }
    }

    printf("¡La tortuga ganadora es la número %d!\n", winner);

    return 0;
}



Overwriting ejem5.c


In [ ]:
!gcc -o ejem5 -fopenmp ejem5.c
%env OMP_NUM_THREADS=6
!time ./ejem5